In [1]:
import xarray as xr
from datetime import date
from datetime import timedelta
import xesmf as xe
import numpy as np
import calendar
import pandas as pd
import geopandas as gpd

ModuleNotFoundError: No module named 'xesmf'

In [ ]:
str1 = "D:/UGP/datasets/cmems_sla_monthly/dt_global_allsat_msla_h_y"
str2 = "_m"
curr = date(1993,1,1)
str3 = ".nc"
str4 = str1 + str(curr.year) + str2 + "0" + str(curr.month) + str3;

sla = pd.DataFrame(columns=["time" ,"indian_ocean" , "arabian_sea" , "bay_of_bengal"])
i = 0
while curr <= date(2019,9,1):
#     print (str4)
    df = xr.open_mfdataset(str4).sel(latitude = slice(0,45) , longitude = slice(45,105)).to_dataframe().reset_index()
    
    indian_ocean = df[(df["latitude"] < 8) & (df["sla"].isnull() == False)]
    arabian_sea = df[(df["latitude"] > 8) & (df["longitude"] < 77.5) & (df["sla"].isnull() == False)]
    bay_bangal = df[(df["latitude"] > 8) & (df["longitude"] > 77.5) & (df["sla"].isnull() == False)]

    s1 = indian_ocean["sla"].mean()
    s2 = arabian_sea["sla"].mean()
    s3 = bay_bangal["sla"].mean()
    sla.loc[i] = [curr,s1,s2,s3]
    i+=1
    
    days_in_month = calendar.monthrange(curr.year, curr.month)[1]
    curr += timedelta(days = days_in_month)
#     print(str4)
    if curr.month < 10:
        str4 = str1 + str(curr.year) + str2 + "0" + str(curr.month) + str3;
    else:
        str4 = str1 + str(curr.year) + str2 + str(curr.month) + str3;


In [34]:
sla.to_csv("D:/UGP/datasets/sla.csv")

In [48]:
region1 = gpd.read_file('D:/UGP/datasets/region1.geojson')
region2 = gpd.read_file('D:/UGP/datasets/region2.geojson')
region3 = gpd.read_file('D:/UGP/datasets/region3.geojson')
region4 = gpd.read_file('D:/UGP/datasets/region4.geojson')


str1 = "D:/UGP/datasets/imd_dataset/_Clim_Pred_LRF_New_RF25_IMD0p25"
curr = 1993
str3 = ".nc"

precip = pd.DataFrame(columns=["time" ,"r1" , "r2" , "r3" , "r4"])
i = 0

while curr <= 2019:
    
    str4 = str1 + str(curr) + str3
    
    df = xr.open_mfdataset(str4).resample(time="1M").reduce(np.nansum).to_dataframe().reset_index()
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'], df['lat']))
    del df
    mask_region1 = (region1.loc[0, 'geometry'])
    mask_region2 = (region2.loc[0, 'geometry'])
    mask_region3 = (region3.loc[0, 'geometry'])
    mask_region4 = (region4.loc[0, 'geometry'])
    gdf.loc[:,'region1'] = gdf.within((region1.loc[0, 'geometry']))
    gdf.loc[:,'region2'] = gdf.within((region2.loc[0, 'geometry']))
    gdf.loc[:,'region3'] = gdf.within((region3.loc[0, 'geometry']))
    gdf.loc[:,'region4'] = gdf.within((region4.loc[0, 'geometry']))
    r1 = pd.DataFrame(gdf[gdf["region1"] == True])
    r2 = pd.DataFrame(gdf[gdf["region2"] == True])
    r3 = pd.DataFrame(gdf[gdf["region3"] == True])
    r4 = pd.DataFrame(gdf[gdf["region4"] == True])
    del gdf
    r1_precip = r1["rf"].groupby(pd.DatetimeIndex(r1['time'])).sum()
    r2_precip = r2["rf"].groupby(pd.DatetimeIndex(r2['time'])).sum()
    r3_precip = r3["rf"].groupby(pd.DatetimeIndex(r3['time'])).sum()
    r4_precip = r4["rf"].groupby(pd.DatetimeIndex(r4['time'])).sum()
    
    temp = pd.concat([r1_precip,r2_precip,r3_precip,r4_precip],1).reset_index()
    temp.columns = ["time",'r1','r2','r3','r4']
    precip = pd.concat([precip , temp])
    curr += 1


In [49]:
precip.to_csv("D:/UGP/datasets/precip.csv")